Demonstrate scan of multiple tones.  Use a simulated resonator.  Use sweep() with a single tone centered on the resonator.

Then exercise Use two tones:  one centered on the resonator and a second tone, off resonance.

In [ ]:
import sys
sys.path.append('../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
# After running demo_00_init.ipynb we decide which bitfile we'd like to use
bitfile = "mkids_2x2_kidsim_v2"
scan = Scan.Scan(bitfile)

In [ ]:
# Initialize simulation chain with one resonance
simu = scan.simuChain
simu.analysis.qout(3)
simu.synthesis.qout(3)
simu.alloff()

# Set mixer.
fMixerSimu = 300
simu.set_mixer_frequency(fMixerSimu)

# Channel separation.
fc = simu.analysis.fc_ch

# Put one resonator 
fResonance = fMixerSimu + 10*fc + 0.1
print("fResonance = {} MHz".format(fResonance))
simu.enable(fResonance)

In [ ]:
# Get the kids chain
chain = scan.kidsChain

# Set the mixer of the analysis and synthesis chains to be the same
#fMixer = 0
#fMixerQ = chain.fq(fMixer)
#_ = scan.set_mixer(fMixerQ)

In [ ]:
chain.sweep?

In [ ]:
chain.analysis.qout(2)
chain.synthesis.qout(2)
fc = 324
df = 20
f,a,phi=chain.sweep(fc-df/2,fc+df/2,N=500,g=0.1, showProgress=False, doProgress=True)

In [ ]:
plt.plot(f,20*np.log10(a/max(a)))
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");


In [ ]:
freqs = np.array([fResonance, fResonance+123.456])
fis = np.array([0.1, 0.2])
gs = np.array([0.4, 0.6])

bandwidth = 20
nf = 500
offsets = get_sweep_offsets(bandwidth, nf)



In [ ]:
chain.set_tones(freqs, fis, gs)
xs = chain.sweep_tones(bandwidth, nf, mean=True, doProgress=True)
for iTone,qFreq in enumerate(chain.qFreqs):
    plt.plot(offsets, np.abs(xs[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Amplitude (ADUs)")
plt.legend()


In [ ]:
chain.set_tones(freqs, fis, gs)
xs2 = chain.sweep_tones(bandwidth, nf, mean=True, doProgress=True)
for iTone,qFreq in enumerate(chain.qFreqs):
    plt.plot(offsets, np.abs(xs2[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Amplitude (ADUs)")
plt.legend()


In [ ]:
for iTone,qFreq in enumerate(chain.qFreqs):
    plt.plot(offsets, np.abs(xs2[:,iTone])-np.abs(xs[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Delta Amplitude (ADUs)")
plt.legend()
